In [360]:
import copy
import math
import random

class Game:
    
    
    def __init__(self,n,k):
        
        # check whether arguments are valid
        if n<5 or k<=2 or 2*k-1>=n:
            raise ValueError('Invalid arguments.')
        self.n = n    
        self.k = k
        
        self.tab = [i+1 for i in range(n)]
        self.properties = [0 for i in range(n)]
        self.players = [[],[]]
        
        self.subsequences = []
        
        
        
        #we start from any number
        for i in range(n-k+1):
            #we get any possible difference between elements in a subsequence
            for r in range(int((n-1)/(k-1))):
                this_subsequence = []
                #we build a subsequence of given length k
                for l in range(k):
                    if i+(r+1)*l<n:
                        this_subsequence.append(self.tab[i+(r+1)*l])
                if len(this_subsequence)==k:
                    self.subsequences.append(this_subsequence)
                    
                    
        # tab contain all players subsequences possible in future,             
        self.possible_players_subsequences = [copy.deepcopy(self.subsequences),copy.deepcopy(self.subsequences)]
        number_all_subsequences = len(self.subsequences)
        
        # tab contains how many numbers of each subsequences payer have chose
        self.score_players_subsequences=[[0 for i in range(number_all_subsequences)],[0 for i in range(number_all_subsequences)]]
        
    def choose_number(self,player,i):
        if self.properties[i-1]!=0:
            print(f"Number {i} is already occupied. Choose another number.")
        else:
            print(f"Player {player} chooses number {i}.")
            self.properties[i-1]=player
            self.players[player-1].append(i)
            print(self.tab)
            print(self.properties)
            
            print(self.players)

            self.update_possible_subsequences_and_scores(player,i)
        
    def update_possible_subsequences_and_scores(self,player,i):
        second_player=self.other_player(player)
            
        # delate possible subsequences in other player wich contain chooses number 
        for j in reversed(range(len(self.possible_players_subsequences[second_player-1]))):
            if i in self.possible_players_subsequences[second_player-1][j]:
               self.possible_players_subsequences[second_player-1].pop(j) 
               self.score_players_subsequences[second_player-1].pop(j)       
                
        
        # update score of possible subsequences, plus one in which contains choose number
        for j in range(len(self.possible_players_subsequences[player-1])):
            if i in self.possible_players_subsequences[player-1][j]:
                self.score_players_subsequences[player-1][j] += 1
                
        print("possible_players_subsequences:")
        print(self.possible_players_subsequences[0])
        print(self.possible_players_subsequences[1])
        
        print("score_players_subsequences:")
        print(self.score_players_subsequences[0])
        print(self.score_players_subsequences[1])
    
    def losing_condition(self,last_player):
        end = False
        for subsequence in self.subsequences:
            if end == True:
                print(f"Player number {last_player} has lost.")
                return 
            this_end = True
            for i in range(len(subsequence)):
                if subsequence[i] not in self.players[last_player-1]:
                    this_end = False
            end = end + this_end
        print("Game has not ended yet.")
        
    def draw_condition(self): #check after losing conditions
        # if no empty field
        if not 0 in self.properties:
            print("Game end, draw") 
            
    def computer_movement(self,player,strategy="defensive"):
        second_player=self.other_player(player)
        
        if (strategy=="defensive"):
            occurrences = [0]*self.n
            for i, j in enumerate(self.properties):
                # fied empty
                if (j==0):
                    # count how many possibles subsequences contains field "j" 
                    for subsequences_i, subsequences in enumerate(self.possible_players_subsequences[player-1]):
                        if i+1 in subsequences:
                            occurrences[i] +=1
                            
                        # if payer have chosen all fields from subsequences apart from
                        #field "j", choose it is loose    
                        if  (self.score_players_subsequences[player-1][subsequences_i] == (self.k - 1)):
                            occurrences[i] = math.inf
                # field not empty    
                else:
                    occurrences[i] = math.inf
            print("occurrences")
            print(occurrences)

            if (min(occurrences) == math.inf):
                # player loose in any case, so we take firts available field
                prefer_field = self.available_fields()[0]

            else :
                # take field which appear at least times possible sequences
                prefer_field= occurrences.index(min(occurrences)) 

            print ("prefer_field:", prefer_field)  

            self.choose_number(player,prefer_field +1 )   
          
        
        elif (strategy=="offensive"):
            occurrences = [0]*self.n
            for i, j in enumerate(self.properties):
                # fied empty
                if (j==0):
                    # count in how many possibles subsequences in second player contains field "j" 
                    for subsequences_i, subsequences in enumerate(self.possible_players_subsequences[second_player-1]):
                        if i+1 in subsequences:
                            occurrences[i] +=1
                            
                        #if payer have chosen all fields from subsequences apart from
                        #field "j", choose it is loose  
                    for subsequences_i, subsequences in enumerate(self.possible_players_subsequences[player-1]):    
                        if  (self.score_players_subsequences[player-1][subsequences_i] == (self.k - 1)):
                           occurrences[i] = math.inf
                # field not empty    
                else:
                    occurrences[i] = math.inf
            print("occurrences")
            print(occurrences)

            if (min(occurrences) == math.inf):
                # player loose in any case, so we take firts available field
                prefer_field = self.available_fields()[0]

            else :
                # take field which appear at least times  in second player possible sequences
                prefer_field= occurrences.index(min(occurrences)) 

            print ("prefer_field:", prefer_field)  

            self.choose_number(player,prefer_field +1 )   
            
            
            
        elif (strategy=="random"):
            available = self.available_fields()
            n = len(available)
            prefer_field =available[random.randint(0, n-1)]
            self.choose_number(player,prefer_field +1 ) 
            
        else : print ("wrong strategy")    
            
    def available_fields(self):
        print("properties")
        print(self.properties)
        field=[]
        for i, j in enumerate(self.properties):
            if (j==0):
                field.append(i)
        return field
    
    def other_player(self,player):
        second_player=2
        if (player == 2):
            second_player=1
        return second_player    
        

In [174]:
g = Game(10,3)
print(g.subsequences)
g.choose_number(1,1)
g.losing_condition(1)
g.draw_condition()

g.computer_movement(2)
g.losing_condition(2)
g.draw_condition()


[[1, 2, 3], [1, 3, 5], [1, 4, 7], [1, 5, 9], [2, 3, 4], [2, 4, 6], [2, 5, 8], [2, 6, 10], [3, 4, 5], [3, 5, 7], [3, 6, 9], [4, 5, 6], [4, 6, 8], [4, 7, 10], [5, 6, 7], [5, 7, 9], [6, 7, 8], [6, 8, 10], [7, 8, 9], [8, 9, 10]]
Player 1 chooses number 1.
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[1], []]
possible_players_subsequences:
[[1, 2, 3], [1, 3, 5], [1, 4, 7], [1, 5, 9], [2, 3, 4], [2, 4, 6], [2, 5, 8], [2, 6, 10], [3, 4, 5], [3, 5, 7], [3, 6, 9], [4, 5, 6], [4, 6, 8], [4, 7, 10], [5, 6, 7], [5, 7, 9], [6, 7, 8], [6, 8, 10], [7, 8, 9], [8, 9, 10]]
[[2, 3, 4], [2, 4, 6], [2, 5, 8], [2, 6, 10], [3, 4, 5], [3, 5, 7], [3, 6, 9], [4, 5, 6], [4, 6, 8], [4, 7, 10], [5, 6, 7], [5, 7, 9], [6, 7, 8], [6, 8, 10], [7, 8, 9], [8, 9, 10]]
score_players_subsequences:
[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Game has not ended yet.
occurrences
[inf, 4, 4, 6, 6, 8, 6, 6, 4, 4]
prefer_field: 1
Player 2 choose

In [175]:
g.choose_number(1,3)
g.losing_condition(1)
g.draw_condition()



Player 1 chooses number 3.
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1, 2, 1, 0, 0, 0, 0, 0, 0, 0]
[[1, 3], [2]]
possible_players_subsequences:
[[1, 3, 5], [1, 4, 7], [1, 5, 9], [3, 4, 5], [3, 5, 7], [3, 6, 9], [4, 5, 6], [4, 6, 8], [4, 7, 10], [5, 6, 7], [5, 7, 9], [6, 7, 8], [6, 8, 10], [7, 8, 9], [8, 9, 10]]
[[2, 4, 6], [2, 5, 8], [2, 6, 10], [4, 5, 6], [4, 6, 8], [4, 7, 10], [5, 6, 7], [5, 7, 9], [6, 7, 8], [6, 8, 10], [7, 8, 9], [8, 9, 10]]
score_players_subsequences:
[2, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Game has not ended yet.


In [176]:
g.computer_movement(2)
g.losing_condition(2)
g.draw_condition()

occurrences
[inf, inf, inf, 4, 4, 7, 5, 6, 3, 4]
prefer_field: 8
Player 2 chooses number 9.
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1, 2, 1, 0, 0, 0, 0, 0, 2, 0]
[[1, 3], [2, 9]]
possible_players_subsequences:
[[1, 3, 5], [1, 4, 7], [3, 4, 5], [3, 5, 7], [4, 5, 6], [4, 6, 8], [4, 7, 10], [5, 6, 7], [6, 7, 8], [6, 8, 10]]
[[2, 4, 6], [2, 5, 8], [2, 6, 10], [4, 5, 6], [4, 6, 8], [4, 7, 10], [5, 6, 7], [5, 7, 9], [6, 7, 8], [6, 8, 10], [7, 8, 9], [8, 9, 10]]
score_players_subsequences:
[2, 1, 1, 1, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1]
Game has not ended yet.


In [177]:
g.choose_number(1,10)
g.losing_condition(1)
g.draw_condition()

Player 1 chooses number 10.
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1, 2, 1, 0, 0, 0, 0, 0, 2, 1]
[[1, 3, 10], [2, 9]]
possible_players_subsequences:
[[1, 3, 5], [1, 4, 7], [3, 4, 5], [3, 5, 7], [4, 5, 6], [4, 6, 8], [4, 7, 10], [5, 6, 7], [6, 7, 8], [6, 8, 10]]
[[2, 4, 6], [2, 5, 8], [4, 5, 6], [4, 6, 8], [5, 6, 7], [5, 7, 9], [6, 7, 8], [7, 8, 9]]
score_players_subsequences:
[2, 1, 1, 1, 0, 0, 1, 0, 0, 1]
[1, 1, 0, 0, 0, 1, 0, 1]
Game has not ended yet.


In [178]:
g.computer_movement(2)
g.losing_condition(2)
g.draw_condition()

occurrences
[inf, inf, inf, 3, 4, 5, 4, 4, inf, inf]
prefer_field: 3
Player 2 chooses number 4.
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1, 2, 1, 2, 0, 0, 0, 0, 2, 1]
[[1, 3, 10], [2, 9, 4]]
possible_players_subsequences:
[[1, 3, 5], [3, 5, 7], [5, 6, 7], [6, 7, 8], [6, 8, 10]]
[[2, 4, 6], [2, 5, 8], [4, 5, 6], [4, 6, 8], [5, 6, 7], [5, 7, 9], [6, 7, 8], [7, 8, 9]]
score_players_subsequences:
[2, 1, 0, 0, 1]
[2, 1, 1, 1, 0, 1, 0, 1]
Game has not ended yet.


In [179]:
g.choose_number(1,6)
g.losing_condition(1)
g.draw_condition()

Player 1 chooses number 6.
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1, 2, 1, 2, 0, 1, 0, 0, 2, 1]
[[1, 3, 10, 6], [2, 9, 4]]
possible_players_subsequences:
[[1, 3, 5], [3, 5, 7], [5, 6, 7], [6, 7, 8], [6, 8, 10]]
[[2, 5, 8], [5, 7, 9], [7, 8, 9]]
score_players_subsequences:
[2, 1, 1, 1, 2]
[1, 1, 1]
Game has not ended yet.


In [180]:
g.computer_movement(2)
g.losing_condition(2)
g.draw_condition()

occurrences
[inf, inf, inf, inf, 2, inf, 2, 2, inf, inf]
prefer_field: 4
Player 2 chooses number 5.
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1, 2, 1, 2, 2, 1, 0, 0, 2, 1]
[[1, 3, 10, 6], [2, 9, 4, 5]]
possible_players_subsequences:
[[6, 7, 8], [6, 8, 10]]
[[2, 5, 8], [5, 7, 9], [7, 8, 9]]
score_players_subsequences:
[1, 2]
[2, 2, 1]
Game has not ended yet.


In [181]:
g.choose_number(1,7)
g.losing_condition(1)
g.draw_condition()

Player 1 chooses number 7.
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1, 2, 1, 2, 2, 1, 1, 0, 2, 1]
[[1, 3, 10, 6, 7], [2, 9, 4, 5]]
possible_players_subsequences:
[[6, 7, 8], [6, 8, 10]]
[[2, 5, 8]]
score_players_subsequences:
[2, 2]
[2]
Game has not ended yet.


In [182]:
g.computer_movement(2)
g.losing_condition(2)
g.draw_condition()

occurrences
[inf, inf, inf, inf, inf, inf, inf, inf, inf, inf]
properties
[1, 2, 1, 2, 2, 1, 1, 0, 2, 1]
prefer_field: 7
Player 2 chooses number 8.
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1, 2, 1, 2, 2, 1, 1, 2, 2, 1]
[[1, 3, 10, 6, 7], [2, 9, 4, 5, 8]]
possible_players_subsequences:
[]
[[2, 5, 8]]
score_players_subsequences:
[]
[3]
Player number 2 has lost.
Game end, draw


In [389]:
random.seed = 123

In [390]:
g = Game(10,3)
print(g.subsequences)
g.computer_movement(1,strategy="random")
g.losing_condition(1)
g.draw_condition()

[[1, 2, 3], [1, 3, 5], [1, 4, 7], [1, 5, 9], [2, 3, 4], [2, 4, 6], [2, 5, 8], [2, 6, 10], [3, 4, 5], [3, 5, 7], [3, 6, 9], [4, 5, 6], [4, 6, 8], [4, 7, 10], [5, 6, 7], [5, 7, 9], [6, 7, 8], [6, 8, 10], [7, 8, 9], [8, 9, 10]]
properties
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Player 1 chooses number 7.
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[[7], []]
possible_players_subsequences:
[[1, 2, 3], [1, 3, 5], [1, 4, 7], [1, 5, 9], [2, 3, 4], [2, 4, 6], [2, 5, 8], [2, 6, 10], [3, 4, 5], [3, 5, 7], [3, 6, 9], [4, 5, 6], [4, 6, 8], [4, 7, 10], [5, 6, 7], [5, 7, 9], [6, 7, 8], [6, 8, 10], [7, 8, 9], [8, 9, 10]]
[[1, 2, 3], [1, 3, 5], [1, 5, 9], [2, 3, 4], [2, 4, 6], [2, 5, 8], [2, 6, 10], [3, 4, 5], [3, 6, 9], [4, 5, 6], [4, 6, 8], [6, 8, 10], [8, 9, 10]]
score_players_subsequences:
[0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Game has not ended yet.


In [391]:
print(g.subsequences)
g.computer_movement(2,strategy="defensive")
g.losing_condition(2)
g.draw_condition()

[[1, 2, 3], [1, 3, 5], [1, 4, 7], [1, 5, 9], [2, 3, 4], [2, 4, 6], [2, 5, 8], [2, 6, 10], [3, 4, 5], [3, 5, 7], [3, 6, 9], [4, 5, 6], [4, 6, 8], [4, 7, 10], [5, 6, 7], [5, 7, 9], [6, 7, 8], [6, 8, 10], [7, 8, 9], [8, 9, 10]]
occurrences
[3, 5, 5, 5, 5, 6, inf, 4, 3, 3]
prefer_field: 0
Player 2 chooses number 1.
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[2, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[[7], [1]]
possible_players_subsequences:
[[2, 3, 4], [2, 4, 6], [2, 5, 8], [2, 6, 10], [3, 4, 5], [3, 5, 7], [3, 6, 9], [4, 5, 6], [4, 6, 8], [4, 7, 10], [5, 6, 7], [5, 7, 9], [6, 7, 8], [6, 8, 10], [7, 8, 9], [8, 9, 10]]
[[1, 2, 3], [1, 3, 5], [1, 5, 9], [2, 3, 4], [2, 4, 6], [2, 5, 8], [2, 6, 10], [3, 4, 5], [3, 6, 9], [4, 5, 6], [4, 6, 8], [6, 8, 10], [8, 9, 10]]
score_players_subsequences:
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0]
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Game has not ended yet.


In [392]:
print(g.subsequences)
g.computer_movement(1,strategy="random")
g.losing_condition(1)
g.draw_condition()

[[1, 2, 3], [1, 3, 5], [1, 4, 7], [1, 5, 9], [2, 3, 4], [2, 4, 6], [2, 5, 8], [2, 6, 10], [3, 4, 5], [3, 5, 7], [3, 6, 9], [4, 5, 6], [4, 6, 8], [4, 7, 10], [5, 6, 7], [5, 7, 9], [6, 7, 8], [6, 8, 10], [7, 8, 9], [8, 9, 10]]
properties
[2, 0, 0, 0, 0, 0, 1, 0, 0, 0]
Player 1 chooses number 10.
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[2, 0, 0, 0, 0, 0, 1, 0, 0, 1]
[[7, 10], [1]]
possible_players_subsequences:
[[2, 3, 4], [2, 4, 6], [2, 5, 8], [2, 6, 10], [3, 4, 5], [3, 5, 7], [3, 6, 9], [4, 5, 6], [4, 6, 8], [4, 7, 10], [5, 6, 7], [5, 7, 9], [6, 7, 8], [6, 8, 10], [7, 8, 9], [8, 9, 10]]
[[1, 2, 3], [1, 3, 5], [1, 5, 9], [2, 3, 4], [2, 4, 6], [2, 5, 8], [3, 4, 5], [3, 6, 9], [4, 5, 6], [4, 6, 8]]
score_players_subsequences:
[0, 0, 0, 1, 0, 1, 0, 0, 0, 2, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
Game has not ended yet.


In [393]:
print(g.subsequences)
g.computer_movement(2,strategy="defensive")
g.losing_condition(2)
g.draw_condition()

[[1, 2, 3], [1, 3, 5], [1, 4, 7], [1, 5, 9], [2, 3, 4], [2, 4, 6], [2, 5, 8], [2, 6, 10], [3, 4, 5], [3, 5, 7], [3, 6, 9], [4, 5, 6], [4, 6, 8], [4, 7, 10], [5, 6, 7], [5, 7, 9], [6, 7, 8], [6, 8, 10], [7, 8, 9], [8, 9, 10]]
occurrences
[inf, 4, 5, 5, 5, 4, inf, 2, 2, inf]
prefer_field: 7
Player 2 chooses number 8.
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[2, 0, 0, 0, 0, 0, 1, 2, 0, 1]
[[7, 10], [1, 8]]
possible_players_subsequences:
[[2, 3, 4], [2, 4, 6], [2, 6, 10], [3, 4, 5], [3, 5, 7], [3, 6, 9], [4, 5, 6], [4, 7, 10], [5, 6, 7], [5, 7, 9]]
[[1, 2, 3], [1, 3, 5], [1, 5, 9], [2, 3, 4], [2, 4, 6], [2, 5, 8], [3, 4, 5], [3, 6, 9], [4, 5, 6], [4, 6, 8]]
score_players_subsequences:
[0, 0, 1, 0, 1, 0, 0, 2, 1, 1]
[1, 1, 1, 0, 0, 1, 0, 0, 0, 1]
Game has not ended yet.


In [394]:
print(g.subsequences)
g.computer_movement(1,strategy="random")
g.losing_condition(1)
g.draw_condition()

[[1, 2, 3], [1, 3, 5], [1, 4, 7], [1, 5, 9], [2, 3, 4], [2, 4, 6], [2, 5, 8], [2, 6, 10], [3, 4, 5], [3, 5, 7], [3, 6, 9], [4, 5, 6], [4, 6, 8], [4, 7, 10], [5, 6, 7], [5, 7, 9], [6, 7, 8], [6, 8, 10], [7, 8, 9], [8, 9, 10]]
properties
[2, 0, 0, 0, 0, 0, 1, 2, 0, 1]
Player 1 chooses number 3.
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[2, 0, 1, 0, 0, 0, 1, 2, 0, 1]
[[7, 10, 3], [1, 8]]
possible_players_subsequences:
[[2, 3, 4], [2, 4, 6], [2, 6, 10], [3, 4, 5], [3, 5, 7], [3, 6, 9], [4, 5, 6], [4, 7, 10], [5, 6, 7], [5, 7, 9]]
[[1, 5, 9], [2, 4, 6], [2, 5, 8], [4, 5, 6], [4, 6, 8]]
score_players_subsequences:
[1, 0, 1, 1, 2, 1, 0, 2, 1, 1]
[1, 0, 1, 0, 1]
Game has not ended yet.


In [395]:
print(g.subsequences)
g.computer_movement(2,strategy="defensive")
g.losing_condition(2)
g.draw_condition()

[[1, 2, 3], [1, 3, 5], [1, 4, 7], [1, 5, 9], [2, 3, 4], [2, 4, 6], [2, 5, 8], [2, 6, 10], [3, 4, 5], [3, 5, 7], [3, 6, 9], [4, 5, 6], [4, 6, 8], [4, 7, 10], [5, 6, 7], [5, 7, 9], [6, 7, 8], [6, 8, 10], [7, 8, 9], [8, 9, 10]]
occurrences
[inf, 2, inf, 3, 3, 3, inf, inf, 1, inf]
prefer_field: 8
Player 2 chooses number 9.
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[2, 0, 1, 0, 0, 0, 1, 2, 2, 1]
[[7, 10, 3], [1, 8, 9]]
possible_players_subsequences:
[[2, 3, 4], [2, 4, 6], [2, 6, 10], [3, 4, 5], [3, 5, 7], [4, 5, 6], [4, 7, 10], [5, 6, 7]]
[[1, 5, 9], [2, 4, 6], [2, 5, 8], [4, 5, 6], [4, 6, 8]]
score_players_subsequences:
[1, 0, 1, 1, 2, 0, 2, 1]
[2, 0, 1, 0, 1]
Game has not ended yet.


In [396]:
print(g.subsequences)
g.computer_movement(1,strategy="random")
g.losing_condition(1)
g.draw_condition()

[[1, 2, 3], [1, 3, 5], [1, 4, 7], [1, 5, 9], [2, 3, 4], [2, 4, 6], [2, 5, 8], [2, 6, 10], [3, 4, 5], [3, 5, 7], [3, 6, 9], [4, 5, 6], [4, 6, 8], [4, 7, 10], [5, 6, 7], [5, 7, 9], [6, 7, 8], [6, 8, 10], [7, 8, 9], [8, 9, 10]]
properties
[2, 0, 1, 0, 0, 0, 1, 2, 2, 1]
Player 1 chooses number 4.
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[2, 0, 1, 1, 0, 0, 1, 2, 2, 1]
[[7, 10, 3, 4], [1, 8, 9]]
possible_players_subsequences:
[[2, 3, 4], [2, 4, 6], [2, 6, 10], [3, 4, 5], [3, 5, 7], [4, 5, 6], [4, 7, 10], [5, 6, 7]]
[[1, 5, 9], [2, 5, 8]]
score_players_subsequences:
[2, 1, 1, 2, 2, 1, 3, 1]
[2, 1]
Player number 1 has lost.


In [67]:
g = Game(10,3)
print(g.subsequences)
g.choose_number(1,1)
g.losing_condition(1)
g.draw_condition()

g.choose_number(2,7)
g.losing_condition(2)
g.choose_number(1,7)
g.choose_number(1,2)
g.losing_condition(1)
g.choose_number(2,9)
g.losing_condition(2)
g.choose_number(1,3)
g.losing_condition(1)

[[1, 2, 3], [1, 3, 5], [1, 4, 7], [1, 5, 9], [2, 3, 4], [2, 4, 6], [2, 5, 8], [2, 6, 10], [3, 4, 5], [3, 5, 7], [3, 6, 9], [4, 5, 6], [4, 6, 8], [4, 7, 10], [5, 6, 7], [5, 7, 9], [6, 7, 8], [6, 8, 10], [7, 8, 9], [8, 9, 10]]
Player 1 chooses number 1.
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[1], []]
possible_players_subsequences:
[[1, 2, 3], [1, 3, 5], [1, 4, 7], [1, 5, 9], [2, 3, 4], [2, 4, 6], [2, 5, 8], [2, 6, 10], [3, 4, 5], [3, 5, 7], [3, 6, 9], [4, 5, 6], [4, 6, 8], [4, 7, 10], [5, 6, 7], [5, 7, 9], [6, 7, 8], [6, 8, 10], [7, 8, 9], [8, 9, 10]]
[[2, 3, 4], [2, 4, 6], [2, 5, 8], [2, 6, 10], [3, 4, 5], [3, 5, 7], [3, 6, 9], [4, 5, 6], [4, 6, 8], [4, 7, 10], [5, 6, 7], [5, 7, 9], [6, 7, 8], [6, 8, 10], [7, 8, 9], [8, 9, 10]]
score_players_subsequences:
[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Game has not ended yet.
Player 2 chooses number 7.
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1, 0, 0, 0, 0, 0

In [361]:
g=Game(10,3)
print(g.subsequences)
g.computer_movement(1,strategy="random")
g.losing_condition(1)
g.draw_condition()

[[1, 2, 3], [1, 3, 5], [1, 4, 7], [1, 5, 9], [2, 3, 4], [2, 4, 6], [2, 5, 8], [2, 6, 10], [3, 4, 5], [3, 5, 7], [3, 6, 9], [4, 5, 6], [4, 6, 8], [4, 7, 10], [5, 6, 7], [5, 7, 9], [6, 7, 8], [6, 8, 10], [7, 8, 9], [8, 9, 10]]
properties
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Player 1 chooses number 5.
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[[5], []]
possible_players_subsequences:
[[1, 2, 3], [1, 3, 5], [1, 4, 7], [1, 5, 9], [2, 3, 4], [2, 4, 6], [2, 5, 8], [2, 6, 10], [3, 4, 5], [3, 5, 7], [3, 6, 9], [4, 5, 6], [4, 6, 8], [4, 7, 10], [5, 6, 7], [5, 7, 9], [6, 7, 8], [6, 8, 10], [7, 8, 9], [8, 9, 10]]
[[1, 2, 3], [1, 4, 7], [2, 3, 4], [2, 4, 6], [2, 6, 10], [3, 6, 9], [4, 6, 8], [4, 7, 10], [6, 7, 8], [6, 8, 10], [7, 8, 9], [8, 9, 10]]
score_players_subsequences:
[0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Game has not ended yet.


In [362]:
g.computer_movement(2,strategy="offensive")
g.losing_condition(2)
g.draw_condition()

occurrences
[4, 5, 6, 7, inf, 8, 7, 6, 5, 4]
prefer_field: 0
Player 2 chooses number 1.
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[2, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[[5], [1]]
possible_players_subsequences:
[[2, 3, 4], [2, 4, 6], [2, 5, 8], [2, 6, 10], [3, 4, 5], [3, 5, 7], [3, 6, 9], [4, 5, 6], [4, 6, 8], [4, 7, 10], [5, 6, 7], [5, 7, 9], [6, 7, 8], [6, 8, 10], [7, 8, 9], [8, 9, 10]]
[[1, 2, 3], [1, 4, 7], [2, 3, 4], [2, 4, 6], [2, 6, 10], [3, 6, 9], [4, 6, 8], [4, 7, 10], [6, 7, 8], [6, 8, 10], [7, 8, 9], [8, 9, 10]]
score_players_subsequences:
[0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Game has not ended yet.


In [363]:
g.computer_movement(1,strategy="random")
g.losing_condition(1)
g.draw_condition()

properties
[2, 0, 0, 0, 1, 0, 0, 0, 0, 0]
Player 1 chooses number 4.
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[2, 0, 0, 1, 1, 0, 0, 0, 0, 0]
[[5, 4], [1]]
possible_players_subsequences:
[[2, 3, 4], [2, 4, 6], [2, 5, 8], [2, 6, 10], [3, 4, 5], [3, 5, 7], [3, 6, 9], [4, 5, 6], [4, 6, 8], [4, 7, 10], [5, 6, 7], [5, 7, 9], [6, 7, 8], [6, 8, 10], [7, 8, 9], [8, 9, 10]]
[[1, 2, 3], [2, 6, 10], [3, 6, 9], [6, 7, 8], [6, 8, 10], [7, 8, 9], [8, 9, 10]]
score_players_subsequences:
[1, 1, 1, 0, 2, 1, 0, 2, 1, 1, 1, 1, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
Game has not ended yet.


In [364]:
g.computer_movement(2,strategy="offensive")
g.losing_condition(2)
g.draw_condition()

occurrences
[inf, 4, 4, inf, inf, 8, 6, 6, 4, 4]
prefer_field: 1
Player 2 chooses number 2.
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[2, 2, 0, 1, 1, 0, 0, 0, 0, 0]
[[5, 4], [1, 2]]
possible_players_subsequences:
[[3, 4, 5], [3, 5, 7], [3, 6, 9], [4, 5, 6], [4, 6, 8], [4, 7, 10], [5, 6, 7], [5, 7, 9], [6, 7, 8], [6, 8, 10], [7, 8, 9], [8, 9, 10]]
[[1, 2, 3], [2, 6, 10], [3, 6, 9], [6, 7, 8], [6, 8, 10], [7, 8, 9], [8, 9, 10]]
score_players_subsequences:
[2, 1, 0, 2, 1, 1, 1, 1, 0, 0, 0, 0]
[2, 1, 0, 0, 0, 0, 0]
Game has not ended yet.


In [365]:
g.computer_movement(1,strategy="random")
g.losing_condition(1)
g.draw_condition()

properties
[2, 2, 0, 1, 1, 0, 0, 0, 0, 0]
Player 1 chooses number 9.
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[2, 2, 0, 1, 1, 0, 0, 0, 1, 0]
[[5, 4, 9], [1, 2]]
possible_players_subsequences:
[[3, 4, 5], [3, 5, 7], [3, 6, 9], [4, 5, 6], [4, 6, 8], [4, 7, 10], [5, 6, 7], [5, 7, 9], [6, 7, 8], [6, 8, 10], [7, 8, 9], [8, 9, 10]]
[[1, 2, 3], [2, 6, 10], [6, 7, 8], [6, 8, 10]]
score_players_subsequences:
[2, 1, 1, 2, 1, 1, 1, 2, 0, 0, 1, 1]
[2, 1, 0, 0]
Game has not ended yet.


In [366]:
g.computer_movement(2,strategy="offensive")
g.losing_condition(2)
g.draw_condition()

occurrences
[inf, inf, inf, inf, inf, inf, inf, inf, inf, inf]
properties
[2, 2, 0, 1, 1, 0, 0, 0, 1, 0]
prefer_field: 2
Player 2 chooses number 3.
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[2, 2, 2, 1, 1, 0, 0, 0, 1, 0]
[[5, 4, 9], [1, 2, 3]]
possible_players_subsequences:
[[4, 5, 6], [4, 6, 8], [4, 7, 10], [5, 6, 7], [5, 7, 9], [6, 7, 8], [6, 8, 10], [7, 8, 9], [8, 9, 10]]
[[1, 2, 3], [2, 6, 10], [6, 7, 8], [6, 8, 10]]
score_players_subsequences:
[2, 1, 1, 1, 2, 0, 0, 1, 1]
[3, 1, 0, 0]
Player number 2 has lost.
